<a href="https://colab.research.google.com/github/upriyam-cmu/EDGE-Rec/blob/main/execute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install project code

In [ ]:
!pip uninstall -y edge-rec
!pip install -e.

# Train model

In [ ]:
from edge_rec.gaussian_diffusion import GaussianDiffusion, Trainer
from edge_rec.model import GraphReconstructionModel

model = GraphReconstructionModel.default()

diffusion_model = GaussianDiffusion(model, image_size=50)
trainer = Trainer(
    diffusion_model, folder="./movie_lens", train_num_steps=int(1e4),
    use_alternate_dense_dataset=True, save_and_sample_every=200,
)
print("Using device:", trainer.device)

In [ ]:
trainer.train()

# Sample ratings

In [ ]:
d70_subgraph, train_ratings, test_ratings = trainer.ds.dataset.get_subgraph(
    subgraph_size=50,
    target_density=0.7,
    include_separate_train_test_ratings=True
)[:-1]
sampled_ratings = trainer.eval(full_graph=d70_subgraph, subgraph_size=50)[0, 0, :, :].cpu().detach()
sampled_ratings = sampled_ratings * 2.5 + 3

# Evaluate metrics

In [ ]:
from edge_rec.evaluate import get_metrics
import torch

train_edges = train_ratings.long().to_sparse_coo()
train_edges = torch.cat([
    train_edges.indices(),
    train_edges.values().unsqueeze(dim=0)
], dim=0)
test_edges = test_ratings.long().to_sparse_coo()
test_edges = torch.cat([
    test_edges.indices(),
    test_edges.values().unsqueeze(dim=0)
], dim=0)

get_metrics(train_edges, test_edges, sampled_ratings)

# Display sampled ratings distribution

In [ ]:
import matplotlib.pyplot as plt

plt.hist(sampled_ratings.numpy().flatten(), bins=5)
plt.show()